# Proyecto - Entrega 1

## Detección de la ocupación de una oficina a partir de datos de luz, temperatura, humedad y concentración de CO2, por medio del uso de modelos estadísticos lineales.

Alejandro Mantilla - 201711304

Ximena Palacio - 201730995 

### Datos

El proyecto se realiza sobre los datos de Luis M. Candanedo, 2015, en los cuales recopilan información de diversos sensores un una oficina para construir una muestra estructurada de 8143 registros para la muestra de entrenamiento, 2665 registros para una muestra de prueba y 9752　para otra muestra de prueba.

Enlace a documento investigativo:

https://doi.org/10.1016/j.enbuild.2015.11.071

El autor del conjunto de datos integra información adquirida de sensores que miden en tiempo real distintos atributos del entorno en una oficina de 5.85m × 3.50m × 3.53m en Bélgica. Particularmente, mide temperatura, humedad, luz y concentración de CO2. Adicionalmente, incluye una variable transformada que llama "Humidity ratio", que calcula a partir de las variables de temperatura y humedad (esta transformación es sugerida por un manual de la sociedad americana de calentadores).

La métrica peculiar de interes en estos datos es un indicador binario de si la oficina está ocupada o no. Podría ser interesante medir la cantidad de personas en la oficina, pero no lo permiten los datos.

La intención del experimento desarrollado por el autor es integrar un sistema detección de la ocupación con controles de energía, ventilación y calentadores para hacer un manejo más eficiente de recursos. El autor reconoce la viabilidad y robustez de un sistema que incorpore cámaras y sistemas de visión de máquina, pero enfatiza la importancia de respetar la privacidad de los usuarios de la oficina.

In [4]:
library(readr)
datos <- read_csv("Archivos/datos_entrenamiento.csv",
                  show_col_types = FALSE)
head(datos)

id,date,Temperature,Humidity,Light,CO2,HumidityRatio,Occupancy
<dbl>,<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,2015-02-04 17:51:00,23.18,27.2720,426.0,721.25,0.004792988,1
2,2015-02-04 17:51:59,23.15,27.2675,429.5,714.00,0.004783441,1
3,2015-02-04 17:53:00,23.15,27.2450,426.0,713.50,0.004779464,1
4,2015-02-04 17:54:00,23.15,27.2000,426.0,708.25,0.004771509,1
5,2015-02-04 17:55:00,23.10,27.2000,426.0,704.50,0.004756993,1
6,2015-02-04 17:55:59,23.10,27.2000,419.0,701.00,0.004756993,1


La variable `date` nos indica el momento en el que se toma el registro. Por el esquema del experimento los registros se toman en tiempo real a intervalos de un minuto aproximadamente. Los datos se registran ininterrumpidadmente a lo largo de 7 días, por lo cual no consideraremos atributos adicionales con los que podríamos enriquecer los datos como factores climáticos, ya que estos son invariantes a lo largo del periodo de muestra. Sugerimos incluir una variable adicional binaria que identifique si la fecha corresponde a un horario laboral.

La variable `Temperature` mide en celcius la temperatura de la oficina.

La variable `Humidity` mide en RH (porcentaje) la humedad relativa en el aire. Aquí 100(%) representa la máxima capacidad de vapor en el aire a la temperatura del momento.

La variable `Light` mide en lux la intensidad de luz en la oficina. Esta variable es influenciada por la iluminación artificial de la oficina y la luz externa.

La variable `CO2` mide en ppm (parts per million) la concentración de CO2 del aire en la oficina.

### De interés.

En la búsqueda de un tema de investigación para el proyecto tuvimos claro que queriamos hallar la intersección entre electrónica asequible (arduino, sensores, sistemas embebidos) y el cuidado del medio ambiente. Este conjunto de datos satisface perfectamente la necesidad de evaluar distintos factores de entorno por medio de tecnología de bajo costo para detectar factores que puedan influenciar sistemas de control. Estos podrían permitir tomar decisiones sobre el uso eficiente de recursos y por consecuente favorecer al cuidado del medio ambiente.

Los autores del dataset sugieren que, aunque existen sistemas de detección altamente capaces, estos suelen depender de sensores que directamente violan la privacidad de los usuarios del espacio como cámaras, micrófonos y seguimiento geo-espacial. Además, el desarrollo de estos sistemas puede significar un costo relvante. La intención es, entonces, el desarrollo de un sistema de bajo costo que abstraiga la necesidad de medir el comportamiento humano y en su lugar detecte factores del entorno.

Por métodos de inferencia estadística queremos identificar la calidad de la información adquirida con el fin de predecir la ocupación de la oficina. Aunque la relación causal usual se encuentre invertida, reconocemos la importancia de diseñar un modelo con alta exactitud y precisión. Incluiremos técnicas de selección y transormación de variables (incluyendo potenciales interacciones) para dedicar el estudio solo a los factores más relevantes y para caracterizar apropiadamente la relación del comportamiento. Utilizaremos también intervalos de confianza con un $\alpha = 0.05$ para validar nuestros hallazgos y para complementar las pruebas de hipótesis.

Algunas pruebas preliminares que buscamos responder son:

- Significancia global y parcial de las variables y sus transformaciones.

- Validación de supuestos y especificación.

- A partir de observaciones no incluidas en el entrenamiento, qué tan acertado es el modelo (pruebas con restricciones).

- Interpretación de los intervalos de confianza de los coeficientes y de combinaciones lineales de los mismos.

### Retos estadísticos

Desde un inicio sabemos que nos enfrentamos a un problema que no debe modelarse como una regresión tradicional por OLS, ya que la naturaleza de los datos puede llevarnos a querer estimar una variable binaria. Tentativamente, utilizaremos modelos lineales generalizados tipo _logit_ y probit.

Definitivamente algunas variables contienen información redundante, por lo que dedicaremos un esfuerzo importante al filtro inicial y a la corrección de supuestos como la multicolinealidad (particularmente por temperatura y humedad). Los datos, además, pueden estar autocorrelacionados espacialmente ya que son datos observados secuencialmente, lo cual deberemos evaluar y corregir de ser necesario.

### Referencias

L.M. Candanedo, V. Feldheim, Accurate occupancy detection of an office room from light, temperature, humidity and CO2 measurements using statistical learning models, Energy and Buildings (2015), recuperado de: http://dx.doi.org/10.1016/j.enbuild.2015.11.071